# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

---
## Лабораторная работа 6.1

### Форматирование строк

6\.1\.1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
import pandas as pd

df = pd.read_csv('recipes_sample.csv')
df.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [ ]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

random_recipes = recipes.sample(n=5)[['id', 'minutes']]

id_width = max(len(str(id_)) for id_ in random_recipes['id'].values)
minutes_width = max(len(str(min_)) for min_ in random_recipes['minutes'].values)

id_width = max(id_width, len('id'))
minutes_width = max(minutes_width, len('minutes'))

header_template = f"|{' id ':^{id_width + 6}}|{'minutes':^{minutes_width + 6}}|"
separator = f"|{'-' * (id_width + 6)}|{'-' * (minutes_width + 6)}|"

print(header_template)
print(separator)
for _, row in random_recipes.iterrows():
    row_template = f"|{str(row['id']):^{id_width + 6}}|{str(row['minutes']):^{minutes_width + 6}}|"
    print(row_template)

|     id     |   minutes   |
|------------|-------------|
|   23842    |     200     |
|   200559   |     75      |
|   183731   |     30      |
|   74392    |     62      |
|   477032   |     55      |


6\.1\.2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [17]:
import pandas as pd
import xml.etree.ElementTree as ET

def capitalize_title(title):
    return ' '.join(word.capitalize() for word in title.split())

def show_info(name, steps, minutes, author_id):
    formatted_name = f'"{capitalize_title(name)}"\n'
    
    formatted_steps = '\n'.join(f"{i+1}. {step.capitalize()}" 
                              for i, step in enumerate(steps))
    separator = "----------"
    author_info = f"Автор: {author_id}"
    time_info = f"Среднее время приготовления: {minutes} минут"
    
    result = f"{formatted_name}\n{formatted_steps}\n{separator}\n{author_info}\n{time_info}\n"
    
    return result

recipes = pd.read_csv('recipes_sample.csv')

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

recipe_data = recipes[recipes['id'] == 170895].iloc[0]

steps = []
target_id = '170895'

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id')
    if recipe_id is not None and recipe_id.text == target_id:
        steps_element = recipe.find('steps')
        if steps_element is not None:
            steps = [step.text for step in steps_element.findall('step')]
        break


result = show_info(
    name=recipe_data['name'],
    steps=steps,
    minutes=recipe_data['minutes'],
    author_id=recipe_data['contributor_id']
)
print(result)


"Leeks And Parsnips Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addit

In [18]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

6\.1\.3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [19]:
import xml.etree.ElementTree as ET
import re
tree = ET.parse('steps_sample.xml')
root = tree.getroot()
time_pattern = r'\d+\s(?:hour|hours|minute|minutes)'

steps = []
target_id = '25082'

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id')
    if recipe_id is not None and recipe_id.text == target_id:
        steps_element = recipe.find('steps')
        if steps_element is not None:
            steps = [step.text for step in steps_element.findall('step')]
        break

print(f"Найденные упоминания времени в рецепте {target_id}:")
for step in steps:
    matches = re.findall(time_pattern, step)
    if matches: 
        print(f"В шаге '{step}':")
        for match in matches:
            print(f"Найдено: {match}")

Найденные упоминания времени в рецепте 25082:
В шаге 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board':
Найдено: 20 minute
В шаге 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly':
Найдено: 10 minute
В шаге 'let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours':
Найдено: 2 hour
В шаге 'when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc':
Найдено: 10 minute
В шаге 'bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom':
Найдено: 20 minute
Найдено: 30 minute


6\.1\.4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [21]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

pattern = r'^this[a-zA-Z0-9_\s]*,\s*but'

matching_recipes = recipes[recipes['description'].str.contains(pattern, regex=True, na=False)]

print(f"Количество рецептов с подходящим шаблоном: {len(matching_recipes)}")
print("\nПримеры описаний:")
print("-" * 80)
for desc in matching_recipes['description'].head(3):
    print(desc)
    print("-" * 80)

Количество рецептов с подходящим шаблоном: 133

Примеры описаний:
--------------------------------------------------------------------------------
this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.
--------------------------------------------------------------------------------
this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.
--------------------------------------------------------------------------------
this is kind of similar to some of the other versions out there, but it is the best and easiest i have fou

## Лабораторная работа 6.2

6\.2\.1\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
import xml.etree.ElementTree as ET
import re

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

steps = []
target_id = '72367'

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id')
    if recipe_id is not None and recipe_id.text == target_id:
        steps_element = recipe.find('steps')
        if steps_element is not None:
            steps = [step.text for step in steps_element.findall('step')]
        break

fraction_pattern = r'(\d+)\s*/\s*(\d+)'

print(f"Шаги рецепта {target_id} после обработки:")
print("-" * 50)

for i, step in enumerate(steps, 1):
    modified_step = re.sub(fraction_pattern, r'\1/\2', step)
    print(f"{i}. {modified_step}")

Шаги рецепта 72367 после обработки:
--------------------------------------------------
1. mix butter , flour , 1/3 c
2. sugar and 1-1/4 t
3. vanilla
4. press into greased 9" springform pan
5. mix cream cheese , 1/4 c
6. sugar , eggs and 1/2 t
7. vanilla beating until fluffy
8. pour over dough
9. combine apples , 1/3 c
10. sugar and cinnamon
11. arrange on top of cream cheese mixture and sprinkle with almonds
12. bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\.2\.2\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [35]:
import pandas as pd
import xml.etree.ElementTree as ET
import nltk
from nltk.tokenize import word_tokenize

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

all_steps = []
for recipe in root.findall('recipe'):
    steps_element = recipe.find('steps')
    if steps_element is not None:
        steps = [step.text for step in steps_element.findall('step')]
        all_steps.extend(steps)
all_text = ' '.join(step for step in all_steps if step is not None)

words = word_tokenize(all_text)

unique_words = set(word.lower() for word in words if word.isalpha())

print(f"Количество уникальных слов в рецептах: {len(unique_words)}")

Количество уникальных слов в рецептах: 14926


6\.2\.3\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [36]:
from nltk.tokenize import sent_tokenize
recipes = pd.read_csv('recipes_sample.csv')

def count_sentences(text):
    if pd.isna(text): 
        return 0
    sentences = sent_tokenize(str(text))
    return len(sentences)
recipes['sentence_count'] = recipes['description'].apply(count_sentences)
longest_descriptions = recipes.nlargest(5, 'sentence_count')

print("5 рецептов с самыми длинными описаниями (по количеству предложений):")
print("\nКоличество предложений в каждом описании:")
for idx, row in longest_descriptions.iterrows():
    print(f"\nID рецепта: {row['id']}")
    print(f"Количество предложений: {row['sentence_count']}")
    print(f"Описание: {row['description']}")
    print("-" * 80)

5 рецептов с самыми длинными описаниями (по количеству предложений):

Количество предложений в каждом описании:

ID рецепта: 334113
Количество предложений: 76
Описание: this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.  it makes a delicious filling also between the layers of cakes and under fondant icing.  you can make roses but it takes 3 or more days to dry them depending on the humidity. 

there are many versions of “buttercream” icing. some are made with eggs and all butter.  some varieties, you have to cook your sugar to a softball stage.  others are 100% shortening or a combination of shortening and butter.

each decorator has his or her favorite.  i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch. but…. i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot 

6\.2\.4\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [38]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')

def display_pos_tags(sentence):
    words = word_tokenize(sentence)

    pos_tags = nltk.pos_tag(words)

    tags_line = ''
    words_line = ''
    
    for i, (word, (_, tag)) in enumerate(zip(words, pos_tags)):
        width = max(len(word), len(tag))

        centered_tag = tag.center(width)
        centered_word = word.center(width)
        tags_line += centered_tag + ' '
        words_line += centered_word + ' '

    print(tags_line.rstrip())
    print(words_line.rstrip())

recipes = pd.read_csv('recipes_sample.csv')

recipe = recipes[recipes['id'] == 241106].iloc[0]
recipe_name = recipe['name']

print("Анализ названия рецепта с id 241106:")
print("-" * 50)
display_pos_tags(recipe_name)

Анализ названия рецепта с id 241106:
--------------------------------------------------
   JJ     NNS    IN     NNS    VBP    JJ    CC   JJ   NNS
eggplant steaks with chickpeas feta cheese and black olives
